In [ ]:
import math
import numpy as np
import pandas as pd
from pandas import Series
#
import matplotlib.pyplot as plt
#
import scipy
from scipy import stats
from scipy.stats import entropy
from scipy.special import rel_entr, kl_div
#
# import matplotlib.gridspec as gridspec
import seaborn as sns
#
from sklearn.metrics import mean_squared_error

In [ ]:
scenario_name_set = ["crossing-90-a01", "crossing-90-a02", "crossing-90-a03", "crossing-90-a04", "crossing-90-a05", 
                     "crossing-90-a06", "crossing-90-a07", "crossing-90-a08", "crossing-90-a09", "crossing-90-a10"]
#
fps_values = [25]
r_contact = 2.0
fps = fps_values[0]
dt = 1.0/fps
#
scenario_name = scenario_name_set[0]
readfile_name = "traj-classification-features_"+scenario_name+".txt"
print ("open", readfile_name)
#
record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
record_raw = record_raw.drop(['direction_i', 'direction_j'], axis=1)
# iterating the columns
for col in record_raw.columns:
    print(col)

In [ ]:
def generate_histograms(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0, x_1, x_2, x_3]
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=16)
            if (j == 0):
                plt.ylabel('frequency', fontsize=16) 
            plt.title(subplot_title[j], fontsize=18)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_Wasserstein_v(scenario_name_set, fps, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    variable_x1 = "v_avg"
    variable_x2 = "v_ini"
    label_x = "contact duration (s)"
    x_min = 0.0
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_max = False
    if not x_max_i:
        check_x_max = True
    #
    print_header = "scenario\tWasserstein\t%_ballistic"
    print(print_header)    
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_a0 = record_raw[variable_x1] # all
        x_a1 = record_raw[variable_x1][(record_raw['direction_ij']==1)] # parallel interaction
        x_a2 = record_raw[variable_x1][(record_raw['direction_ij']==2)] # head-on interaction
        x_a3 = record_raw[variable_x1][(record_raw['direction_ij']==3)] # crossing interaction
        x_a = [x_a0, x_a1, x_a2, x_a3]
        #
        x_b0 = record_raw[variable_x2] # all
        x_b1 = record_raw[variable_x2][(record_raw['direction_ij']==1)] # parallel interaction
        x_b2 = record_raw[variable_x2][(record_raw['direction_ij']==2)] # head-on interaction
        x_b3 = record_raw[variable_x2][(record_raw['direction_ij']==3)] # crossing interaction
        x_b = [x_b0, x_b1, x_b2, x_b3]      
        #
        count_ballistic = len(record_raw[variable_x1][(record_raw['motion_type']=="B")]) # ballistic motion)
        #
        subplot_title = []
        #subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append(scenario_name_set[i])
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")
        #
        if (check_x_max == True):
            x_max = max(x_a[0])
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        print_wasserstein = scenario_name_set[i]
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            # bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth)          
            # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            counts1 = plt.hist(x_a[j], bins=bin_list, weights=np.ones_like(x_a[j])/len(x_a[j]), alpha=0.5) # alpha controls transparency
            counts2 = plt.hist(x_b[j], bins=bin_list, weights=np.ones_like(x_b[j])/len(x_b[j]), histtype="step", linewidth=3)
            #
            wasserstein_value = stats.wasserstein_distance(counts1[0], counts2[0])
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=16)
            if (j == 0):
                plt.ylabel('frequency', fontsize=16) 
                #
                print_wasserstein += "\t{:.4f}".format(wasserstein_value)
            plt.title(subplot_title[j], fontsize=18) 
        print_wasserstein += "\t{:.2f}".format(count_ballistic/len(x_a0)*100.0)
        print(print_wasserstein)
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_relative(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0, x_1, x_2, x_3]
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i            
        #
        n_xticks = 5
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('relative frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_deg(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i = "", x_max_i = ""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True           
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0/np.pi*180, x_1/np.pi*180, x_2/np.pi*180, x_3/np.pi*180] # deg
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")  
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i    
        #
        xtick_space = 45
        if (variable_x == "alpha_std"):
            xtick_space = 30  
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) # x_min, x_max, x_binwidth are given in deg
            # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
            #
            plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))
            plt.xlim(x_min, x_max)              
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_histograms_relative_deg(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i = "", x_max_i = ""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True           
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0/np.pi*180, x_1/np.pi*180, x_2/np.pi*180, x_3/np.pi*180] # deg
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")  
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i    
        #
        xtick_space = 45
        if (variable_x == "alpha_std"):
            xtick_space = 30  
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            #
            bin_list = np.arange(x_min, x_max*1.05, x_binwidth) # x_min, x_max, x_binwidth are given in deg
            counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
            #
            plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))
            plt.xlim(x_min, x_max)              
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel('relative frequency', fontsize=12) 
            plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_scatterplot(scenario_name_set, fps, 
                         variable_x, variable_y, label_x, label_y,
                         x_min_i="", x_max_i="", xtick_space="", 
                         y_min_i="", y_max_i="", ytick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.5
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    check_y_min = False
    check_y_max = False
    if not y_min_i:
        check_y_min = True
    if not y_max_i:
        check_y_max = True        
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0)     
        #
        # x axis
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0, x_1, x_2, x_3]
        #
        if (variable_x in ["alpha_avg", "turning_angle_avg"]):
            x = [x_0/np.pi*180, x_1/np.pi*180, x_2/np.pi*180, x_3/np.pi*180] # deg                 
        #
        # y axis
        #
        y_0 = record_raw[variable_y] # all
        y_1 = record_raw[variable_y][(record_raw['direction_ij']==1)] # parallel interaction
        y_2 = record_raw[variable_y][(record_raw['direction_ij']==2)] # head-on interaction
        y_3 = record_raw[variable_y][(record_raw['direction_ij']==3)] # crossing interaction        
        y = [y_0, y_1, y_2, y_3]   
        #
        if (variable_y in ["alpha_avg", "turning_angle_avg"]):
            y = [y_0/np.pi*180, y_1/np.pi*180, y_2/np.pi*180, y_3/np.pi*180] # deg               
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")  
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i  
        #
        if (check_y_min == True):
            y_min = min(min(y[0]), 0.0)  
        else: 
            y_min = y_min_i
        if (check_y_max == True):
            y_max = max(y[0])  
        else: 
            y_max = y_max_i  
        #        
        n_xticks = 4
        n_yticks = 4
        #
        for j in range (0, n_columns):
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            plt.scatter(x[j], y[j], alpha=0.1, s=2)
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))               
            #
            plt.ylim(y_min, y_max)
            if not ytick_space:
                ax.yaxis.set_major_locator(plt.MaxNLocator(n_yticks))
            else:
                plt.yticks(np.arange(y_min, y_max*1.05, ytick_space))                  
            #
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel(label_y, fontsize=12)
            plt.title(subplot_title[j], fontsize=16)
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()

In [ ]:
def generate_heatmap(scenario_name_set, fps, 
                         variable_x, variable_y, label_x, label_y,
                         x_min_i="", x_max_i="", xtick_space="", 
                         y_min_i="", y_max_i="", ytick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_columns = n_types+1
    n_rows = len(scenario_name_set)
    subplot_x = 4.0
    subplot_y = 3.5
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    check_y_min = False
    check_y_max = False
    if not y_min_i:
        check_y_min = True
    if not y_max_i:
        check_y_max = True        
    #
    for i in range (0, len(scenario_name_set)):
        dt = 1.0/fps
        scenario_name = scenario_name_set[i]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        # x axis
        #
        x_0 = record_raw[variable_x] # all
        x_1 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_2 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_3 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x = [x_0, x_1, x_2, x_3]
        #
        if (variable_x == "n_neighbors_avg"):
            x = [x_0/(r_contact*r_contact*np.pi), x_1/(r_contact*r_contact*np.pi), x_2/(r_contact*r_contact*np.pi), x_3/(r_contact*r_contact*np.pi)]
        if (variable_x == "n_neighbors_std"):
            x = [x_0/(r_contact*r_contact*np.pi), x_1/(r_contact*r_contact*np.pi), x_2/(r_contact*r_contact*np.pi), x_3/(r_contact*r_contact*np.pi)]   
        if (variable_x == "frame_start"):
            x = [x_0/fps, x_1/fps, x_2/fps, x_3/fps]
        if (variable_x == "frame_end"):
            x = [x_0/fps, x_1/fps, x_2/fps, x_3/fps]   
        if (variable_x == "alpha_avg"):
            x = [x_0/np.pi*180, x_1/np.pi*180, x_2/np.pi*180, x_3/np.pi*180] # deg
        if (variable_x == "turning_angle_avg"):
            x = [x_0/np.pi*180, x_1/np.pi*180, x_2/np.pi*180, x_3/np.pi*180] # deg              
        #
        # y axis
        #
        y_0 = record_raw[variable_y] # all
        y_1 = record_raw[variable_y][(record_raw['direction_ij']==1)] # parallel interaction
        y_2 = record_raw[variable_y][(record_raw['direction_ij']==2)] # head-on interaction
        y_3 = record_raw[variable_y][(record_raw['direction_ij']==3)] # crossing interaction        
        y = [y_0, y_1, y_2, y_3]       
        #
        if (variable_y == "alpha_avg"):
            y = [y_0/np.pi*180, y_1/np.pi*180, y_2/np.pi*180, y_3/np.pi*180] # deg  
        if (variable_y == "turning_angle_avg"):
            y = [y_0/np.pi*180, y_1/np.pi*180, y_2/np.pi*180, y_3/np.pi*180] # deg             
        #
        subplot_title = []
        subplot_title.append(scenario_name_set[i]+" (all)")
        subplot_title.append("(parallel)")
        subplot_title.append("(head-on)")
        subplot_title.append("(crossing)")   
        #
        if (check_x_min == True):
            x_min = min(min(x[0]), 0.0)  
        else: 
            x_min = x_min_i
        if (check_x_max == True):
            x_max = max(x[0])  
        else: 
            x_max = x_max_i  
        #
        if (check_y_min == True):
            y_min = min(min(y[0]), 0.0)  
        else: 
            y_min = y_min_i
        if (check_y_max == True):
            y_max = max(y[0])  
        else: 
            y_max = y_max_i  
        #        
        n_xticks = 4
        n_yticks = 4
        #
        # print(type(x[1]), type(y[1]))
        for j in range (0, n_columns):
            #
            x_series = pd.Series(x[j])
            y_series = pd.Series(y[j])
            df_input = pd.DataFrame({'x': x_series,'y': y_series})    
            df_input = df_input.dropna()
            #
            ax = fig.add_subplot(n_rows,n_columns,(n_columns*i+j+1))
            plt.hist2d(df_input['x'], df_input['y'], bins=(20, 20), range=[[x_min, x_max*1.05],[y_min, y_max*1.05]], cmap=plt.cm.Reds)          
            #
            plt.xlim(x_min, x_max)
            if not xtick_space:
                ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
            else:
                plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))               
            #
            plt.ylim(y_min, y_max)
            if not ytick_space:
                ax.yaxis.set_major_locator(plt.MaxNLocator(n_yticks))
            else:
                plt.yticks(np.arange(y_min, y_max*1.05, ytick_space))                  
            #
            plt.tick_params(labelsize=12)
            if (i == (len(scenario_name_set)-1)):
                plt.xlabel(label_x, fontsize=12)
            if (j == 0):
                plt.ylabel(label_y, fontsize=12)
            plt.title(subplot_title[j], fontsize=16)
            plt.colorbar()
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.4)    
    #   
    plt.show()

In [ ]:
def generate_histograms_classification(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = (n_types+1)*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types+1):
            row_id = scenario_index*(n_types+1)+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_0 = record_raw[variable_x] # all
            subplot_title_0 = scenario_name
            subplot_title_1 = ""
            if (i == 1):
                # parallel interaction 
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
                subplot_title_0 = "parallel"
            if (i == 2):
                # head-on interaction 
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
                subplot_title_0 = "head-on"
            if (i == 3):
                # crossing interaction
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
                subplot_title_0 = "crossing"
            # subplot_title_0 = subplot_title_0 +" ("+ subplot_title_1 +")"
            #
            x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
            x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
            x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x = [x_0, x_1, x_2, x_3]
            #
            subplot_title = []
            subplot_title.append(subplot_title_0)
            if (i == 0):
                subplot_title.append(subplot_title_1+"ballistic")
                subplot_title.append(subplot_title_1+"confined")
                subplot_title.append(subplot_title_1+"sub-ballistic")
            else:
                subplot_title.append(subplot_title_0+" & ballistic")
                subplot_title.append(subplot_title_0+" & confined")
                subplot_title.append(subplot_title_0+" & sub-ballistic")                
            #
            if (check_x_min == True):
                x_min = min(min(x[0]), 0.0)  
            else: 
                x_min = x_min_i
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5
            #
            for j in range(0, n_columns):
                if (len(x[j]) > 0):                
                    ax = fig.add_subplot(n_rows, n_columns, (n_columns*row_id+j+1))
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    # counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                    counts = plt.hist(x[j], bins=bin_list, alpha=0.5) # alpha controls transparency
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=14)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=16)
                    if (j == 0):
                        plt.ylabel('frequency', fontsize=16) 
                    plt.title(subplot_title[j], fontsize=18)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_Wasserstein_classification_v(scenario_name_set, fps, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    variable_x1 = "v_avg"
    variable_x2 = "v_ini"
    label_x = "contact duration (s)"
    x_min = 0.0
    #
    # prepare subplots
    #
    n_types = 3
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = (n_types+1)*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_max = False
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    print_wasserstein_header = "\tall\tballistic\tconfined\tsub-ballistic"
    print(print_wasserstein_header)    
    #
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types+1):
            row_id = scenario_index*(n_types+1)+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_a0 = record_raw[variable_x1] # all
            x_b0 = record_raw[variable_x2] # all
            subplot_title_0 = scenario_name
            subplot_title_1 = ""
            if (i == 0):
                subplot_title_1 = "all"
            if (i == 1):
                # parallel interaction 
                x_a0 = record_raw[variable_x1][(record_raw['direction_ij']==1)] # parallel interaction
                x_b0 = record_raw[variable_x2][(record_raw['direction_ij']==1)] # parallel interaction
                subplot_title_1 = "parallel"
            if (i == 2):
                # head-on interaction 
                x_a0 = record_raw[variable_x1][(record_raw['direction_ij']==2)] # head-on interaction
                x_b0 = record_raw[variable_x2][(record_raw['direction_ij']==2)] # head-on interaction
                subplot_title_1 = "head-on"
            if (i == 3):
                # crossing interaction
                x_a0 = record_raw[variable_x1][(record_raw['direction_ij']==3)] # crossing interaction
                x_b0 = record_raw[variable_x2][(record_raw['direction_ij']==3)] # crossing interaction
                subplot_title_1 = "crossing"
            #
            x_a1 = x_a0[(record_raw['motion_type']=="B")] # ballistic motion
            x_a2 = x_a0[(record_raw['motion_type']=="C")] # confined motion
            x_a3 = x_a0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x_a = [x_a0, x_a1, x_a2, x_a3]
            #
            x_b1 = x_b0[(record_raw['motion_type']=="B")] # ballistic motion
            x_b2 = x_b0[(record_raw['motion_type']=="C")] # confined motion
            x_b3 = x_b0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x_b = [x_b0, x_b1, x_b2, x_b3]            
            #
            # subplot_title_0 = subplot_title_0 +" ("+ subplot_title_1 +")"
            subplot_title_0 = subplot_title_0 +" ("+ subplot_title_1 +")"
            subplot_title = []
            subplot_title.append(subplot_title_0)
            subplot_title.append(subplot_title_1+" & ballistic")
            subplot_title.append(subplot_title_1+" & confined")
            subplot_title.append(subplot_title_1+" & sub-ballistic")
            #
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5
            #
            print_wasserstein = subplot_title_0            
            #
            for j in range(0, n_columns):
                if (len(x_a[j]) > 0):                
                    ax = fig.add_subplot(n_rows, n_columns, (n_columns*row_id+j+1))
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    counts1 = plt.hist(x_a[j], bins=bin_list, weights=np.ones_like(x_a[j])/len(x_a[j]), alpha=0.5) # alpha controls transparency
                    counts2 = plt.hist(x_b[j], bins=bin_list, weights=np.ones_like(x_b[j])/len(x_b[j]), histtype="step", linewidth=3) # alpha controls transparency
                    #
                    wasserstein_value = stats.wasserstein_distance(counts1[0], counts2[0]) 
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=12)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=12)
                    if (j == 0):
                        plt.ylabel('relative frequency', fontsize=12) 
                    plt.title(subplot_title[j], fontsize=16)    
                    #
                    print_wasserstein += "\t{:.4f}".format(wasserstein_value)
                    # print(subplot_title[j], "{:.4f}".format(wasserstein_value))
                else:
                    # len(x_a[j]) == 0
                    wasserstein_value = np.nan
                    print_wasserstein += "\t"+str(wasserstein_value)
            print(print_wasserstein)                    
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def generate_histograms_classification_relative(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min_i="", x_max_i="", xtick_space=""):
    #
    # prepare subplots
    #
    n_types = 3
    n_classifications = 3
    n_scenarios = len(scenario_name_set)
    #
    n_columns = n_classifications+1
    n_rows    = (n_types+1)*n_scenarios
    #
    subplot_x = 4.0
    subplot_y = 3.0
    figsize_x = subplot_x*n_columns
    figsize_y = subplot_y*n_rows
    fig = plt.figure(figsize=(figsize_x, figsize_y))       
    #
    check_x_min = False
    check_x_max = False
    if not x_min_i:
        check_x_min = True
    if not x_max_i:
        check_x_max = True
    #
    # for i in range (0, len(scenario_name_set)):
    for scenario_index in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[scenario_index]
        for i in range(0, n_types+1):
            row_id = scenario_index*(n_types+1)+i
            dt = 1.0/fps
            # scenario_name = scenario_name_set[i]
            readfile_name = "traj-classification-features_"+scenario_name+".txt"
            record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
            #
            x_0 = record_raw[variable_x] # all
            subplot_title_0 = scenario_name
            subplot_title_1 = ""
            if (i == 0):
                subplot_title_1 = "all"
            if (i == 1):
                # parallel interaction 
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
                subplot_title_1 = "parallel"
            if (i == 2):
                # head-on interaction 
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
                subplot_title_1 = "head-on"
            if (i == 3):
                # crossing interaction
                x_0 = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
                subplot_title_1 = "crossing"
            subplot_title_0 = subplot_title_0 +" ("+ subplot_title_1 +")"
            #
            x_1 = x_0[(record_raw['motion_type']=="B")] # ballistic motion
            x_2 = x_0[(record_raw['motion_type']=="C")] # confined motion
            x_3 = x_0[(record_raw['motion_type']=="S")] # sub-ballistic motion
            x = [x_0, x_1, x_2, x_3]
            #
            subplot_title = []
            subplot_title.append(subplot_title_0)
            subplot_title.append(subplot_title_1+" :: ballistic")
            subplot_title.append(subplot_title_1+" :: confined")
            subplot_title.append(subplot_title_1+" :: sub-ballistic")
            #
            if (check_x_min == True):
                x_min = min(min(x[0]), 0.0)  
            else: 
                x_min = x_min_i
            if (check_x_max == True):
                x_max = max(x[0])  
            else: 
                x_max = x_max_i            
            #
            n_xticks = 5
            #
            for j in range(0, n_columns):
                if (len(x[j]) > 0):                
                    ax = fig.add_subplot(n_rows, n_columns, (n_columns*row_id+j+1))
                    bin_list = np.arange(x_min, x_max*1.05, x_binwidth) 
                    counts = plt.hist(x[j], bins=bin_list, alpha=0.5, weights=np.ones_like(x[j])/len(x[j])) # alpha controls transparency
                    #
                    plt.xlim(x_min, x_max)
                    if not xtick_space:
                        ax.xaxis.set_major_locator(plt.MaxNLocator(n_xticks))
                    else:
                        plt.xticks(np.arange(x_min, x_max*1.05, xtick_space))       
                    plt.tick_params(labelsize=12)
                    if (row_id == (n_rows-1)):
                        plt.xlabel(label_x, fontsize=12)
                    if (j == 0):
                        plt.ylabel('relative frequency', fontsize=12) 
                    plt.title(subplot_title[j], fontsize=16)               
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
    plt.show()        

In [ ]:
def show_classification_count(scenario_name_set):
    #
    print_header = "\tNo_contacts\t\t\t"
    print_header += "\nscenario\tcount_all\tall_ballistic\tall_confined\tall_sub-ballistic"
    print_header += "\tcount_parallel\tparallel_ballistic\tparallel_confined\tparallel_sub-ballistic"
    print_header += "\tcount_head-on\thead-on_ballistic\thead-on_confined\thead-on_sub-ballistic"
    print_header += "\tcount_crossing\tcrossing_ballistic\tcrossing_confined\tcrossing_sub-ballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        variable_x = "tc_data"
        print_count = scenario_name
        print_ratio = ""
        #
        x_all      = record_raw[variable_x] # all
        x_parallel = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_headon   = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_crossing = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x_base = [x_all, x_parallel, x_headon, x_crossing]
        #
        n_classifications = 3
        for j in range(0, len(x_base)):
            count_ballistic    = len(x_base[j][(record_raw['motion_type']=="B")]) # ballistic motion
            count_confined     = len(x_base[j][(record_raw['motion_type']=="C")]) # confined motion
            count_subballistic = len(x_base[j][(record_raw['motion_type']=="S")]) # sub-ballistic motion
            count_subtotal     = count_ballistic+count_confined+count_subballistic
            print_count += "\t"+str(count_subtotal)+"\t"+str(count_ballistic)+"\t"+str(count_confined)+"\t"+str(count_subballistic)
            print_ratio += "\t"
            print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_confined/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_subballistic/count_subtotal*100)                
        #
        print(print_count)
        print(print_ratio)

In [ ]:
def show_classification_summary(scenario_name_set, variable_x):
    #
    print_header = "scenario\tcategory\t_all\tall_ballistic\tall_confined\tall_subballistic"
    print_header += "\t_parallel\tparallel_ballistic\tparallel_confined\tparallel_subballistic"
    print_header += "\t_head-on\thead-on_ballistic\thead-on_confined\thead-on_subballistic"
    print_header += "\t_crossing\tcrossing_ballistic\tcrossing_confined\tcrossing_subballistic"
    print(print_header)
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all      = record_raw[variable_x] # all
        x_parallel = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_headon   = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_crossing = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x_base = [x_all, x_parallel, x_headon, x_crossing]
        #
        n_classifications = 3
        print_count = scenario_name+"\tcount"
        print_ratio = "\tratio_sub%"
        print_avg = "\t"+variable_x+"_avg"
        print_std = "\t"+variable_x+"_std"
        for j in range(0, len(x_base)):
            x_subtotal     = x_base[j]
            x_ballistic    = x_base[j][(record_raw['motion_type']=="B")] # ballistic motion
            x_confined     = x_base[j][(record_raw['motion_type']=="C")] # confined motion
            x_subballistic = x_base[j][(record_raw['motion_type']=="S")] # sub-ballistic motion
            #
            count_subtotal     = len(x_subtotal)
            count_ballistic    = len(x_ballistic)
            count_confined     = len(x_confined)
            count_subballistic = len(x_subballistic)
            #
            avg_subtotal     = np.mean(x_subtotal)
            avg_ballistic    = np.mean(x_ballistic)
            avg_confined     = np.mean(x_confined)
            avg_subballistic = np.mean(x_subballistic)
            #
            std_subtotal     = np.std(x_subtotal)
            std_ballistic    = np.std(x_ballistic)
            std_confined     = np.std(x_confined)            
            std_subballistic = np.std(x_subballistic)   
            #
            print_count += "\t"+str(count_subtotal)+"\t"+str(count_ballistic)+"\t"+str(count_confined)+"\t"+str(count_subballistic)
            #
            print_ratio += "\t"
            print_ratio += "\t{:.2f}".format(count_ballistic/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_confined/count_subtotal*100)
            print_ratio += "\t{:.2f}".format(count_subballistic/count_subtotal*100)  
            #
            print_avg += "\t{:.2f}".format(avg_subtotal)
            print_avg += "\t{:.2f}".format(avg_ballistic)
            print_avg += "\t{:.2f}".format(avg_confined)
            print_avg += "\t{:.2f}".format(avg_subballistic) 
            #
            print_std += "\t{:.2f}".format(std_subtotal)
            print_std += "\t{:.2f}".format(std_ballistic)
            print_std += "\t{:.2f}".format(std_confined)
            print_std += "\t{:.2f}".format(std_subballistic)             
        #
        print(print_count)
        print(print_ratio)
        print(print_avg)
        print(print_std)

In [ ]:
def count_parallel(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_parallel\tall_ballistic\tall_confined"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_parallel  = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_ballistic = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        x_confined0 = record_raw[variable_x][(record_raw['motion_type']=="C")] # confined motion
        x_confined1 = record_raw[variable_x][(record_raw['direction_ij']==1)&(record_raw['motion_type']=="C")] # confined motion in parallel interaction
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_parallel))+"\t"+str(len(x_ballistic))
        print_count += "\t"+str(len(x_confined0))+"\t"+str(len(x_confined1))
        print(print_count)                

In [ ]:
def count_crossing(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_parallel\tall_crossing\tall_ballistic\tcrossing_ballistic\tall_confined\tparallel_confined"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all        = record_raw[variable_x] # all
        x_parallel   = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_crossing   = record_raw[variable_x][(record_raw['direction_ij']==3)] # parallel interaction
        x_ballistic0 = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        x_ballistic1 = record_raw[variable_x][(record_raw['direction_ij']==3)&(record_raw['motion_type']=="B")] # ballistic motion in crossing interaction
        x_confined0  = record_raw[variable_x][(record_raw['motion_type']=="C")] # confined motion
        x_confined1  = record_raw[variable_x][(record_raw['direction_ij']==1)&(record_raw['motion_type']=="C")] # confined motion in parallel interaction
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_parallel))+"\t"+str(len(x_crossing))
        print_count += "\t"+str(len(x_ballistic0))+"\t"+str(len(x_ballistic1))
        print_count += "\t"+str(len(x_confined0))+"\t"+str(len(x_confined1))
        print(print_count)        

In [ ]:
def count_ballistic(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_head-on\tall_crossing\tall_ballistic"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_headon    = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_crossing  = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        x_ballistic = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_headon))+"\t"+str(len(x_crossing))+"\t"+str(len(x_ballistic))
        print(print_count)

In [ ]:
def check_count_tc(scenario_name_set):
    #
    print_header = "scenario\t_all\tall_parallel\tall_ballistic\tall_confined\ttc_confined\ttc_all"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_parallel  = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_ballistic = record_raw[variable_x][(record_raw['motion_type']=="B")] # ballistic motion
        x_confined  = record_raw[variable_x][(record_raw['motion_type']=="C")] # confined motion
        tc_confined = np.mean(x_confined)
        tc_all      = np.mean(x_all)
        #
        print_count = scenario_name+"\t"+str(len(x_all))+"\t"+str(len(x_parallel))+"\t"+str(len(x_ballistic))
        print_count += "\t"+str(len(x_confined))+"\t{:.1f}".format(tc_confined)+"\t{:.1f}".format(tc_all)
        print(print_count)         

In [ ]:
def basic_statistics(scenario_name_set):
    #
    print_header = "scenario\t_parallel\t_%\t_head-on\t_%\t_crossing\t_%\t_all"
    print(print_header)
    variable_x = "tc_data"
    #     
    for row_id in range(0, len(scenario_name_set)):
        scenario_name = scenario_name_set[row_id]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        print_summary = scenario_name
        #
        x_all       = record_raw[variable_x] # all
        x_parallel  = record_raw[variable_x][(record_raw['direction_ij']==1)] # parallel interaction
        x_headon    = record_raw[variable_x][(record_raw['direction_ij']==2)] # head-on interaction
        x_crossing  = record_raw[variable_x][(record_raw['direction_ij']==3)] # crossing interaction
        #
        print_count = scenario_name
        print_count += "\t"+str(len(x_parallel))+" ({:.2f}".format(len(x_parallel)*100/len(x_all))+"\%)"
        print_count += " & "+str(len(x_headon))+" ({:.2f}".format(len(x_headon)*100/len(x_all))+"\%)"
        print_count += " & "+str(len(x_crossing))+" ({:.2f}".format(len(x_crossing)*100/len(x_all))+"\%)"
        print_count += " & "+str(len(x_all))
        print(print_count)                

In [ ]:
def generate_annotated_heatmaps_individuals(scenario_name_set):
    #
    # creating annotated heatmaps
    # see tips: 
    # https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    # https://matplotlib.org/stable/gallery/color/colorbar_basics.html
    # 
    variable_x = "tc_data"
    for k in range (0, len(scenario_name_set)):
        # x-axis: motion type -- ballistic, confined, sub-ballistic
        # y-axis: contact interaction types -- parallel, head-on, crossing        
        #
        scenario_name = scenario_name_set[k]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        interaction_types = ["parallel", "head-on", "crossing"]
        interaction_values = [1, 2, 3]
        motion_types = ["ballistic", "confined", "sub-ballistic"]
        motion_values = ["B", "C", "S"]
        #        
        count_classification = []
        for i in range(0, len(interaction_types)):    
            count_i = []
            for j in range(0, len(motion_types)):
                count_ij = len(record_raw[variable_x][(record_raw['direction_ij']==interaction_values[i])&(record_raw['motion_type']==motion_values[j])])
                count_i.append(count_ij)
                # count_ij = len(record_raw[variable_x])
                ## print(scenario_name_set[k], interaction_types[i], motion_types[j], count_ij)
            ## print(scenario_name_set[k], count_i)
            count_classification.append(count_i)
        #
        count_classification = np.array(count_classification)
        ratio_classification = count_classification/np.sum(count_classification)
        threshold = count_classification.max()*0.8
        #
        figsize_x = 6.0
        figsize_y = 4.0
        fig, ax = plt.subplots(figsize=(figsize_x, figsize_y))
        im = ax.imshow(ratio_classification, cmap = "YlGn", interpolation='none', vmin=0, vmax=1)

        # Show all ticks and label them with the respective list entries
        ax.set_xticks(np.arange(len(motion_types)))
        ax.set_yticks(np.arange(len(interaction_types)))

        # Rotate the tick labels and set their alignment.
        plt.setp(ax, xticklabels = motion_types, yticklabels = interaction_types)
        plt.xticks(fontsize=16, rotation=45)
        plt.yticks(fontsize=16)
        
        # Loop over data dimensions and create text annotations
        """
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                color_ij = "black"
                if (count_classification[j, i] > threshold): color_ij = "white"
                text = ax.text(i, j, count_classification[j, i], ha="center", va="center", size=16, color=color_ij, fontweight="bold") 
        """
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                # text = ax.text(i, j, count_classification[j, i], ha="center", va="center", size=16, fontweight="bold")
                text = ax.text(i, j, count_classification[j, i], ha="center", va="center", size=16)
        
        #
        ax.set_title(scenario_name_set[k], fontsize = 20)
        fig.tight_layout()
        plt.show()        

In [ ]:
def generate_annotated_heatmaps_avg(scenario_name_set, variable_x):
    #
    # creating annotated heatmaps
    # see tips: 
    # https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    # https://matplotlib.org/stable/gallery/color/colorbar_basics.html
    # 
    # variable_x = "tc_data"
    for k in range (0, len(scenario_name_set)):
        # x-axis: motion type -- ballistic, confined, sub-ballistic
        # y-axis: contact interaction types -- parallel, head-on, crossing        
        #
        scenario_name = scenario_name_set[k]
        readfile_name = "traj-classification-features_"+scenario_name+".txt"
        record_raw = pd.read_csv(readfile_name, sep= ",", header=0) 
        #
        interaction_types = ["parallel", "head-on", "crossing"]
        interaction_values = [1, 2, 3]
        motion_types = ["ballistic", "confined", "sub-ballistic"]
        motion_values = ["B", "C", "S"]
        #        
        avg_classification = []
        for i in range(0, len(interaction_types)):    
            avg_i = []
            for j in range(0, len(motion_types)):
                x_ij = record_raw[variable_x][(record_raw['direction_ij']==interaction_values[i])&(record_raw['motion_type']==motion_values[j])]
                avg_ij = np.mean(x_ij)
                avg_i.append(avg_ij)
            avg_classification.append(avg_i)
        #
        avg_classification = np.array(avg_classification)
        threshold = np.nanmax(avg_classification)*0.7
        # print(scenario_name_set[k], "{:.1f}".format(np.nanmax(avg_classification)), "{:.1f}".format(threshold))
        #
        figsize_x = 5.0
        figsize_y = 3.0
        # fig, ax = plt.subplots()
        fig, ax = plt.subplots(figsize=(figsize_x, figsize_y))
        if (k >= len(scenario_name_set)-3):
            fig, ax = plt.subplots()
        # im = ax.imshow(avg_classification, cmap = "YlGn", interpolation='none', vmin=0, vmax=1)
        im = ax.imshow(avg_classification, cmap = "YlGn", interpolation='none', vmin=0)

        # Show all ticks and label them with the respective list entries
        ax.set_xticks(np.arange(len(motion_types)))
        ax.set_yticks(np.arange(len(interaction_types)))

        # Rotate the tick labels and set their alignment.
        # plt.setp(ax, xticklabels = motion_types, yticklabels = interaction_types)
        #
        plt.setp(ax, xticklabels = "", yticklabels = "")
        if (k%3 == 0):
            # plt.setp(ax, yticklabels = interaction_types)
            plt.yticks([0, 1, 2], interaction_types)
        if (k >= len(scenario_name_set)-3):
            # plt.setp(ax, xticklabels = motion_types)
            plt.xticks([0, 1, 2], motion_types)        
        #
        plt.xticks(fontsize=16, rotation=45)
        plt.yticks(fontsize=16)
        
        # Loop over data dimensions and create text annotations
        
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                color_ij = "black"
                if (avg_classification[j, i] > threshold): color_ij = "white"
                text = ax.text(i, j, "{:.1f}".format(avg_classification[j, i]), ha="center", va="center", size=16, color=color_ij, fontweight="bold")    
        """
        for i in range(0, len(motion_types)):
            for j in range(0, len(interaction_types)):    
                text = ax.text(i, j, "{:.1f}".format(avg_classification[j, i]), ha="center", va="center", size=16)     
        """
        #
        ax.set_title(scenario_name_set[k], fontsize = 20)
        fig.tight_layout()
        plt.show()        

In [ ]:
variable_x = "n_datapoints"
label_x = "No. position data points"
x_binwidth = 10
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
variable_x = "tc_assumed"
label_x = "ballistic contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
variable_x = "turning_angle_entropy"
label_x = "turning angle entropy"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "efficiency"
label_x = "efficiency"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
generate_histograms(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "turning_angle_entropy"
label_x = "turning angle entropy"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
#
generate_histograms_classification(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "efficiency"
label_x = "efficiency"
x_binwidth = 0.05
x_min = 0.0
x_max = 1.0
#
generate_histograms_classification(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min, x_max)

In [ ]:
variable_x = "tc_data"
label_x = "contact duration (s)"
x_binwidth = 0.5
x_min = 0.0
#
generate_histograms_classification(scenario_name_set, fps_values[0], variable_x, label_x, x_binwidth, x_min)

In [ ]:
variable_x = "v_avg"
label_x = "average contact speed (m/s)"
x_binwidth = 0.1
x_min = 0.0
x_max = ""
xtick_space = ""
generate_histograms_classification(scenario_name_set, fps, variable_x, label_x, x_binwidth, x_min, x_max, xtick_space)

In [ ]:
#
show_classification_count(scenario_name_set)

In [ ]:
variable_x = "tc_data"
show_classification_summary(scenario_name_set, variable_x)

In [ ]:
count_parallel(scenario_name_set)

In [ ]:
count_crossing(scenario_name_set)

In [ ]:
count_ballistic(scenario_name_set)

In [ ]:
check_count_tc(scenario_name_set)

In [ ]:
basic_statistics(scenario_name_set)

In [ ]:
generate_annotated_heatmaps_individuals(scenario_name_set)

In [ ]:
generate_annotated_heatmaps_avg(scenario_name_set, "tc_data")